# VideoRobot Colab Runner 🖥️

این نوت‌بوک تمام مراحل لازم برای نصب پیش‌نیازها، اجرای بک‌اند و ارسال یک کار رندر نمونه را به‌صورت خودکار فراهم می‌کند.

> اگر قبلاً مخزن را کلون نکرده‌اید، ابتدا دستور `!git clone https://github.com/englishpodcasteasy-glitch/videorobot.git` را اجرا کنید و سپس دایرکتوری کاری را به همان مسیر تغییر دهید.


In [ ]:
# نصب یا تأیید FFmpeg
!bash scripts/install_ffmpeg_colab.sh

In [ ]:
# نصب وابستگی‌های پایتون
!pip install -q -r backend/requirements.txt

In [ ]:
import os
os.environ.setdefault('BACKEND_PORT', '8000')
print('BACKEND_PORT =', os.environ['BACKEND_PORT'])

In [ ]:
# اجرای بک‌اند در پس‌زمینه
import subprocess, sys
backend = subprocess.Popen([sys.executable, 'backend/main.py'])
backend

In [ ]:
# بررسی سلامت سرویس
import time, requests
last_exc = None
for _ in range(30):
    try:
        resp = requests.get('http://127.0.0.1:8000/healthz', timeout=2)
        if resp.ok:
            print('Backend is healthy:', resp.json())
            break
    except Exception as exc:
        last_exc = exc
        time.sleep(1)
else:
    raise SystemExit(f'Backend did not become healthy in time: {last_exc}')

In [ ]:
# ساخت مانیفست نمونه
import json
from pathlib import Path
manifest = {
    'seed': 42,
    'video': {'width': 720, 'height': 1280, 'fps': 30, 'bg_color': '#101318'},
    'tracks': [
        {'type': 'text', 'content': 'سلام دنیا', 'start': 0.2, 'duration': 3, 'x': 60, 'y': 80, 'size': 72, 'color': '#FFFFFF'},
    ],
}
manifest_path = Path('/content/manifest.json')
manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding='utf-8')
manifest_path

In [ ]:
# ارسال درخواست رندر
import json, requests
with open('/content/manifest.json', 'r', encoding='utf-8') as fh:
    payload = json.load(fh)
resp = requests.post('http://127.0.0.1:8000/render', json=payload, timeout=10)
resp.raise_for_status()
job_info = resp.json()
job_info

In [ ]:
# پیگیری وضعیت تا اتمام
import time, requests
job_id = job_info['data']['job_id']
while True:
    progress = requests.get(f'http://127.0.0.1:8000/progress/{job_id}', timeout=10).json()
    state = progress['data']['state']
    pct = progress['data']['pct']
    print(f'State: {state} | pct={pct}')
    if state in {'success', 'error'}:
        break
    time.sleep(2)
progress

In [ ]:
# دانلود فایل خروجی (در Colab در /content/outputs نگه داشته می‌شود)
from pathlib import Path
result_path = Path(progress['data']['result']['mp4'])
print('Result path:', result_path)
print('Exists:', result_path.exists())

In [ ]:
# توقف فرایند بک‌اند در پایان نوت‌بوک
backend.terminate()
backend.wait(timeout=10)
print('Backend stopped')